### Pre-process pictures for *train*
1. compute **nmf** for all wav files and save as `npy` file
2. get **labels** as ground truth from `wav` files path and save as `npy` file
3. save paths of all `npy` file in `train.h5` and `val.h5`

In [ ]:
import os
import numpy as np
from dataset.NMF import audio_import

#indexes1 = ['accordion', 'acoustic_guitar', 'cello',  'flute',  'saxophone', 'trumpet', 'violin', 'xylophone']
#indexes2 = [[401], [402], [486], [558], [776], [513], [889], [642]]
#indexes3 = [[401], [402,546], [486], [558], [776], [513], [889], [642]]
idxs=[401,402,486,513,558,642,776,889]
names=['accordion','acoustic_guitar','cello','trumpet','flute','xylophone','saxophone','violin']

path = 'h:/Study/bpfile/dataset/audios/'
wav_files = []
bases_filepath = []
labels_filepath = []

# 遍历数据集 `path` 路径下所有的 wav 文件，记录他们的路径
# wav_files[0] = 'h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/1.wav'
for root,dirs,files in os.walk(path, topdown=True):
    for name in files:
        if '.wav' in name:
            wav_files.append(os.path.normcase(root+os.path.sep+name).replace('\\','/'))
            
# 计算 bases 和 labels
for i, fname in enumerate(wav_files):
    label = np.zeros(1000)   # 1000个乐器类别
    
    # 每个 wav 对应的 NMF 结果保存为 *.npy 文件，对应路径存入 bases
    # bases_filepath[0] = 'h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/1_bases.npy'
    # np.array(nmf_base[0]).shape = (2401, 16)
    nmf_base = audio_import.nmf([fname])
    np.save(fname[:-4]+'_bases'+'.npy', nmf_base[0])
    bases_filepath.append(fname[:-4]+'_bases'+'.npy')
    
    # 根据文件路径名确定当前音频中乐器的 label，作为 ground truth 训练
    # labels_filepath[0] = 'h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/1_labels.npy'
    for instrument, index in zip(names, idxs):
        if instrument in fname:
            label[index] = 1
    print(sum(label))
    np.save(fname[:-4]+'_labels'+'.npy', label)
    labels_filepath.append(fname[:-4]+'_labels'+'.npy')
    
    print(bases_filepath[-1])
    print(labels_filepath[-1])
    

### generate `train.h5` and `val.h5`

In [ ]:
import h5py
import random

dataset_num = len(wav_files)
train_num = int(0.7*dataset_num)

# 随机选取 70% 的数据作为验证集
train_bases = random.sample(range(dataset_num), train_num)

# 路径 utf-8 编码，否则报错
train_bases_encode = []
train_labels_encode = []
val_bases_encode = []
val_labels_encode = []
for i in range(dataset_num):
    if i in train_bases:
        train_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
        train_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))
    else:
        val_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
        val_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))

h5f = h5py.File('dataset/train.h5', 'w')
h5f.create_dataset('bases', data=train_bases_encode)
h5f.create_dataset('labels', data=train_labels_encode)
h5f.close()

h5f = h5py.File('dataset/val.h5', 'w')
h5f.create_dataset('bases', data=val_bases_encode)
h5f.create_dataset('labels', data=val_labels_encode)
h5f.close()

## train ...

### Pre-process pictures for *test*
1. compute **nmf** for all wav files and save as `npy` file
2. compute **labels** by feat_extrator and save as `npy` file
3. save paths of all `npy` file in `train.h5` and `val.h5`

In [6]:
import os
import random
import numpy as np
from dataset.NMF import audio_import
from feat_extractor import load_model, get_CAM, feat_pred

idxs=[401,402,486,513,558,642,776,889]
names=['accordion','acoustic_guitar','cello','trumpet','flute','xylophone','saxophone','violin']

test_path = 'h:/Study/bpfile/testset25/'
wav_files = []
img_files = []
bases_filepath = []
labels_filepath = []
locations = {}

# 寻找音频路径与图片路径
# wav_dir = 'h:/Study/bpfile/testset25/testaudio/'
# img_dir = 'h:/Study/bpfile/testset25/testimage/'
wav_dir = os.path.normcase(os.path.join(test_path, 'gt_audio/')).replace('\\','/')
img_dir = os.path.normcase(os.path.join(test_path, 'testimage/')).replace('\\','/')
'''
for cur_dir in os.listdir(test_path):
    if 'audio' in cur_dir:
        os.path.normcase(os.path.join(test_path, cur_dir, '/')).replace('\\','/')
    if 'image' in cur_dir:
        os.path.normcase(os.path.join(test_path, cur_dir, '/')).replace('\\','/')
'''
        
# 寻找所有 wav 的文件名、图片文件所在文件夹
# wav_files[0] = 'accordion_1_saxophone_1.wav'
# img_files[0] = 'accordion_1_saxophone_1'
for f in os.listdir(wav_dir):   # 剔除非 wav 文件
    if '_gt1.wav' not in f and '_gt2.wav' not in f and '.wav' in f:
        wav_files.append(f)
for f in os.listdir(img_dir):   # 剔除目录下所有文件，只保留文件夹
    if os.path.isdir(img_dir + f):
        img_files.append(f)
        locations[f+'.mp4'] = []
        
# 排序确保 wav 文件与图片文件夹相对应
wav_files.sort()
img_files.sort()
load_model()

# 计算 bases 和 labels
for wav_fname, img_folder in zip(wav_files, img_files):
    assert wav_fname[:-4] == img_folder    # 确保 wav 文件与图片文件夹相对应
    
    # 计算 NMF 并将结果保存在 npy 文件中
    # bases_filepath[0] = 'h:/Study/bpfile/testset25/testaudio/accordion_1_saxophone_1_bases.npy'
    # np.array(nmf_base[0]).shape = (2401, 16)
    nmf_base = audio_import.nmf([wav_dir+wav_fname])
    np.save(wav_dir+wav_fname[:-4]+'_bases'+'.npy', nmf_base[0])
    bases_filepath.append(wav_dir+wav_fname[:-4]+'_bases'+'.npy')
    
    imgs = []
    # 获得当前文件夹下所有图片（绝对）路径，并随机抽样
    # img_folder = 'accordion_1_saxophone_1'
    # imgs[0] = '000001.jpg'
    for img in os.listdir(img_dir+img_folder):
        if '.jpg' in img or '.png' in img:
            imgs.append(img)
    imgs = random.sample(imgs, 30)  # 由于图片数量过多，每个视频中只随机抽取 50 张图片进行预测
    
    probs=np.zeros([8])
    location = np.zeros([8])
    # 计算图像的 label，并获得每种乐器的定位（列号）
    for img in imgs:
        probs1 = feat_pred(img_dir+img_folder, img)
        probs = probs + np.array(probs1)
        #(locate, CAMs, heatmap) = get_CAM(img_dir+img_folder, 'results', img)  # heatmap 保存为文件
        locate = get_CAM(img_dir+img_folder, 'results', img)
        location = location + locate
        '''
        print(np.argmax(CAMs))
        print(CAMs)
        plt.figure()
        plt.imshow(0.3*CAMs)
        plt.figure()
        plt.imshow(heatmap)
        '''
        
    locations[img_folder+'.mp4'] = location
    print(probs)
    print(location)
    softmax = np.zeros(1000)
    for i in range(len(probs)):
        softmax[idxs[i]] = probs[i]
    np.save(img_dir+img_folder+'/labels'+'.npy', softmax)
    labels_filepath.append(img_dir+img_folder+'/labels'+'.npy')
    
    print(bases_filepath[-1])
    print(labels_filepath[-1])
    

d:\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


[2.96628118e+01 1.14671146e-03 4.29135547e-04 1.21046786e-04
 1.62240299e-03 4.02420901e-04 8.94150823e-04 2.92536615e-04]
[41639. 31178. 25732. 22913. 25359. 29263. 39276. 22865.]
h:/study/bpfile/testset25/gt_audio/accordion_1_saxophone_1_bases.npy
h:/study/bpfile/testset25/testimage/accordion_1_saxophone_1/labels.npy
[5.67647554e+00 1.92098907e+01 2.25566893e-02 1.15542082e-03
 3.09670911e-03 3.56995286e-03 1.24098510e-02 2.79464242e-02]
[44922. 23471. 49251. 33075. 29141. 43558. 36062. 47445.]
h:/study/bpfile/testset25/gt_audio/accordion_2_acoustic_guitar_2_bases.npy
h:/study/bpfile/testset25/testimage/accordion_2_acoustic_guitar_2/labels.npy
[2.99497473e+01 2.57470918e-03 2.64813239e-02 1.56515756e-04
 9.66574507e-04 1.09977279e-04 1.54362612e-03 4.05431414e-03]
[ 7218.  8681. 14898. 10493. 15633. 11106.  8446. 13478.]
h:/study/bpfile/testset25/gt_audio/accordion_2_cello_3_bases.npy
h:/study/bpfile/testset25/testimage/accordion_2_cello_3/labels.npy
[1.72320598e+01 7.85301381e+00 7.

In [8]:
import h5py

test_bases_encode = []
test_labels_encode = []

for i in range(len(bases_filepath)):
    test_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
    test_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))

h5f = h5py.File('dataset/test.h5', 'w')
h5f.create_dataset('bases', data=test_bases_encode)
h5f.create_dataset('labels', data=test_labels_encode)
h5f.close()

In [9]:
import numpy as np

np.save('dataset/locations'+'.npy', locations)